In [77]:
from utils.seed_setter import set_seed
set_seed()

In [78]:
import pickle,os

with open(os.path.join('./pickle_data/train_test_data/test_data.pickle'), 'rb') as f:
    x_test, y_test = pickle.load(f)
f.close()

In [79]:
from utils.black_box import BlackBox
black_box = BlackBox()

In [80]:
all_preds = black_box.predict_all(x_test)

In [81]:
adversarial_list = []

attack_list = []

for sent, lab, pred in list(zip(x_test, y_test, all_preds)):
        if round(pred) == lab and len(sent) < 500:
            attack_list += [(sent,lab)]
        elif round(pred) != lab and len(sent) < 500:
            adversarial_list += [(sent,lab)]

In [82]:
len(attack_list)

1610

In [83]:
len(adversarial_list)

144

In [84]:
attack_list[:3]

[('This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.',
  0),
 ("The trailers get you to the movie, but the movie just wasn't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they hadn't added this one to their resume's. To be honest, the book is better..",
  0),
 ('The message of this movie is "personality is more important than beauty". Jeanine Garofalo is supposed to be the "ugly duckling", but the funny thing is that she\'s not at all ugly (actually she\'s a lot more attractive than Uma Thurman, the friend who looks like a model).Now, would this movie work if the "ugly duckling" was really unattractive? When will Hollywood stop with this hypocrisy?In my opinion, despite the m

In [85]:
print('Accuracy for the attack list:')
black_box.evaluate([sent for sent, lab in attack_list], [lab for sent, lab in attack_list])

Accuracy for the attack list:
1610/1610 [==============================] - 4s 3ms/sample - loss: 0.0466 - acc: 1.0000


[0.0465955188139255, 1.0]

In [86]:
print('Accuracy for the adversarial list:')
black_box.evaluate([sent for sent, lab in adversarial_list], [lab for sent, lab in adversarial_list])

Accuracy for the adversarial list:
144/144 [==============================] - 0s 3ms/sample - loss: 2.2604 - acc: 0.0000e+00


[2.2603875266181097, 0.0]

In [87]:
#del black_box

In [88]:
'''def most_similar(word, delta = 0.5, num_words = 20):
    
    try:
        index = tokens_dictionary[word]
    except:
        return []
    
    if (index > distance_matrix.shape[0]):
        return []
    
    dist_order = np.argsort(distance_matrix[index,:])[1:num_words+1]
    dist_list = distance_matrix[index][dist_order]
    
    mask = np.ones_like(dist_list)
    mask = np.where(dist_list < delta)
    return [inverse_tokens_dictionary[index] for index in dist_order[mask]]'''

'def most_similar(word, delta = 0.5, num_words = 20):\n    \n    try:\n        index = tokens_dictionary[word]\n    except:\n        return []\n    \n    if (index > distance_matrix.shape[0]):\n        return []\n    \n    dist_order = np.argsort(distance_matrix[index,:])[1:num_words+1]\n    dist_list = distance_matrix[index][dist_order]\n    \n    mask = np.ones_like(dist_list)\n    mask = np.where(dist_list < delta)\n    return [inverse_tokens_dictionary[index] for index in dist_order[mask]]'

In [94]:
import string
import re
import os
import numpy as np
import pickle
import licensed_scripts.lm_1b_eval as google_language_model_utils
import nltk
from utils.black_box import BlackBox

class Attack(object):
    
    def __init__ (self):
        with open(os.path.join('./pickle_data/attack_utils/tokens_dictionary.pickle'), 'rb') as f:
            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)
            self.__tokens_dictionary = tokens_dictionary
            self.__inverse_tokens_dictionary = inverse_tokens_dictionary
        f.close()
        
        self.__black_box = BlackBox()
        self.__distance_matrix = np.load(os.path.join('./numpy_files/distance_matrix.npy'))
        self.__google_lm = google_language_model_utils.LM()
        self.__stopwords = nltk.corpus.stopwords.words('english')
        self.__sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        self.__tree_bank_word_tokenizer = nltk.tokenize.TreebankWordTokenizer()
        
        latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
        safe_characters = string.ascii_letters + string.digits + latin_similar + ' '
        safe_characters += "'"
        self.__safe_characters = safe_characters
        
        self.__max_words = 5 #10
        self.__top_neighbours = 8 #20
        self.__top_lm = 4
        self.__max_gens = 10
        self.__max_children = 5
        
        
    def __handle_contractions(self, sentence):
        sentence = self.__tree_bank_word_tokenizer.tokenize(sentence)
        return ' '.join(sentence)
    
    def __preprocess_sentence(self, sentence):
        chars = set(w for w in sentence)
        symbols = [c for c in chars if not c in self.__safe_characters]
        for symbol in symbols:
            new_sentence = sentence.replace(symbol, ' ' + symbol + ' ')
        new_sentence = self.__handle_contractions(new_sentence)
        return new_sentence.split()
    
    def __split_review(self, review):
        sentences = self.__sentence_tokenizer.tokenize(review)
        for index, sentence in enumerate(sentences):
            sentences[index] = self.__preprocess_sentence(sentence)
        return sentences
            
    def __most_similar(self, word, delta = 0.5, num_words = 20):
        try:
            index = self.__tokens_dictionary[word]
        except:
            return []

        if (index > self.__distance_matrix.shape[0]):
            return []

        dist_order = np.argsort(self.__distance_matrix[index,:])[1:num_words+1]
        dist_list = self.__distance_matrix[index][dist_order]

        mask = np.ones_like(dist_list)
        mask = np.where(dist_list < delta)
        return [self.__inverse_tokens_dictionary[index] for index in dist_order[mask]]         
    
    def __rejoin_review(self, sentences):
        new_sentences = sentences[:]
        for i, sent in enumerate (new_sentences):
            new_sentence = ' '.join(sent)
            new_sentence = re.sub(r' ([^A-Za-z0-9])', r'\1', new_sentence)
            new_sentences[i] = new_sentence
        return ' '.join(new_sentences)
    
    def __perturb(self, sentences, word_index, neighbours, y_target, changed_words_list = []):
        
        print('Starting perturbation:')
        
        prefix = ' '.join(sentences[word_index[0]][ : word_index[1]])
        suffix = ' '.join(sentences[word_index[0]][word_index[1]+1 : -1])
        
        if self.__top_neighbours > len(neighbours):
            top_neighbours = len(neighbours)
        else:
            top_neighbours = self.__top_neighbours
        
        lm_preds = self.__google_lm.get_words_probs(
            prefix, 
            neighbours[ : top_neighbours - 1], 
            suffix
        )
        
        score_list = []
        for adv_w in np.argsort(lm_preds)[-self.__top_lm : ]: 
            adv_sentences = sentences[:]
            adv_splitted_text = adv_sentences[word_index[0]][:]
            adv_splitted_text[word_index[1]] = neighbours[adv_w]
            adv_sentences[word_index[0]] = adv_splitted_text
            adv_review = self.__rejoin_review(adv_sentences)
            score = self.__black_box.predict_sentiment(adv_review)
            score_list += [(adv_review, score, changed_words_list + [word_index])]
            #print(adv_review)
            #print('Score:' + str(score) + '\n')
            
        adv_reviews_sorted =  sorted(score_list, key=lambda x: x[1])    
        
        print('Final review:')
        
        if y_target == 0:
            print(adv_reviews_sorted[0][0] + '\n')
            print('Score:' + str(adv_reviews_sorted[0][1]) + '\n')
            return adv_reviews_sorted[0]
            
        else:
            print(adv_reviews_sorted[-1][0] + '\n')
            print('Score:' + str(adv_reviews_sorted[-1][1]) + '\n')
            return adv_reviews_sorted[-1]
        
        
    def __crossover(self, parent1, parent2):
        splitted_parent1 = self.__split_review(parent1[0])
        splitted_parent2 = self.__split_review(parent2[0])
        changed_word_list = []
        for i in range(len(splitted_parent1)):
            for j in range(len(splitted_parent1[i])):
                if np.random.uniform() < 0.5:
                    splitted_parent1[i][j] = splitted_parent2[i][j]
                    if (i,j) in parent2[2]:
                        changed_word_list += [(i,j)]
                elif (i,j) in parent2[2]:
                    changed_word_list += [(i,j)]
        #print(splitted_parent1)
        return self.__rejoin_review(splitted_parent1), changed_word_list
    
    def __get_neighbours_dictionary_and_words_to_change(self, sentences, changed_word_list = []):
        neighbours_dictionary = {}
        for sent_idx, sent in enumerate(sentences):
            for word_idx, word in enumerate(sent):
                neighbours_dictionary[(sent_idx, word_idx)] = self.__most_similar(
                    word = word.lower(), delta = 0.5, num_words = 50
                )

        neighbours_length = {key: len(value) for key, value in neighbours_dictionary.items()}

        for key in neighbours_length.keys():
            if sentences[key[0]][key[1]].lower() in self.__stopwords or key in changed_word_list:
                neighbours_length[key] = 0

        length_sum = sum(neighbours_length.values())

        neighbours_length = {key: value/length_sum for key, value in neighbours_length.items()}

        probabilities = list(neighbours_length.values())

        random_choice_size = len(np.nonzero(probabilities)[0]) if len(np.nonzero(probabilities)[0]) < self.__max_words else self.__max_words

        words_to_change = np.random.choice(
            len(neighbours_length.keys()), size = random_choice_size, replace = False, p = probabilities
        )
        
        return neighbours_dictionary, words_to_change

    
        
    def attack (self, x_orig, y_orig):
        y_target = int(not y_orig)
        sentences = self.__split_review(x_orig)
        
        neighbours_dictionary, words_to_change = self.__get_neighbours_dictionary_and_words_to_change(sentences)
        
        #print (words_to_change)
        
        print('Original sentence: \n' + x_orig + '\n')
        
        print('Generating population... \n')

        population = [self.__perturb(
                sentences, 
                list(neighbours_dictionary.keys())[index], 
                neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                y_target
            ) for index in words_to_change]
            
        for i in range(self.__max_gens):

            if len(population) == 0:
                return None
            
            print('Generation #{}: \n'.format(i))
            
            sorted_population =  sorted(population, key=lambda x: x[1])    

            #print('Final review:')

            if y_target == 0:
                best_attack = sorted_population[0]
            else:
                best_attack = sorted_population[-1]
                
            print('Best Adversarial:\n {} \n Score: {}\n'.format(best_attack[0], best_attack[1]))

            if round(best_attack[1]) == y_target:
                return best_attack[0]
        
            best_adversarials = [best_attack]
            
            pop_scores = np.array([score for review, score, _ in sorted_population])
            
            #print('Pop scores: ' + str(pop_scores))
            
            #logits = np.exp(pop_scores - np.max(pop_scores))
            logits = np.exp(pop_scores / 0.3)
            selection_probabilities = logits / np.sum(logits)
            
            #print('Probab: ')
            #print(selection_probabilities)
            
            if y_target == 0:
                selection_probabilities = selection_probabilities[::-1]
                #print('New probab:')
                #print(selection_probabilities)
                
            parent_list_1 = np.random.choice(
                len(sorted_population), size=self.__max_children, p=selection_probabilities)
            parent_list_2 = np.random.choice(
                len(sorted_population), size=self.__max_children, p=selection_probabilities)
            
            children = [self.__crossover(
                sorted_population[parent_list_1[i]], 
                sorted_population[parent_list_2[i]]
            ) for i in range(self.__max_children)]
            
            perturbated_children = []
            
            print('########################################\n\n')
            print('Regenerating population \n')
            
            for child in children:
                
                ######################################DUPLICATED CODE##############################################
                sentences = self.__split_review(child[0])
                
                neighbours_dictionary, words_to_change = self.__get_neighbours_dictionary_and_words_to_change(sentences, child[1])
                
                ######################################END DUPLICATED CODE##############################################
                if (len(words_to_change)):
                    perturbated_children += [self.__perturb(
                        sentences, 
                        list(neighbours_dictionary.keys())[index], 
                        neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                        y_target,
                        child[1]
                    ) for index in words_to_change]
            
            population = [best_attack] + perturbated_children
            
        return None

In [95]:
'''import string
import os
import numpy as np
import pickle
import licensed_scripts.lm_1b_eval as google_language_model_utils
import nltk
from utils.black_box import BlackBox

class Attack(object):
    
    def __init__ (self):
        with open(os.path.join('./pickle_data/attack_utils/tokens_dictionary.pickle'), 'rb') as f:
            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)
            self.__tokens_dictionary = tokens_dictionary
            self.__inverse_tokens_dictionary = inverse_tokens_dictionary
        f.close()
        
        self.__black_box = BlackBox()
        self.__distance_matrix = np.load(os.path.join('./numpy_files/distance_matrix.npy'))
        self.__google_lm = google_language_model_utils.LM()
        self.__stopwords = nltk.corpus.stopwords.words('english')
        self.__sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        self.__tree_bank_word_tokenizer = nltk.tokenize.TreebankWordTokenizer()
        
        latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
        safe_characters = string.ascii_letters + string.digits + latin_similar + ' '
        safe_characters += "'"
        self.__safe_characters = safe_characters
        
        self.__max_words = 5 #10
        self.__top_neighbours = 5 #20
        self.__top_lm = 5
        self.__max_gens = 100
        self.__max_children = 4
        
        
    def __handle_contractions(self, sentence):
        sentence = self.__tree_bank_word_tokenizer.tokenize(sentence)
        return ' '.join(sentence)
    
    def __preprocess_sentence(self, sentence):
        chars = set(w for w in sentence)
        symbols = [c for c in chars if not c in self.__safe_characters]
        for symbol in symbols:
            sentence = sentence.replace(symbol, ' ' + symbol + ' ')
        sentence = self.__handle_contractions(sentence)
        return sentence.split()
    
    def __split_review(self, review):
        sentences = self.__sentence_tokenizer.tokenize(review)
        for index, sentence in enumerate(sentences):
            sentences[index] = self.__preprocess_sentence(sentence)
        return sentences
            
            
    def __most_similar(self, word, delta = 0.5, num_words = 20):
        try:
            index = self.__tokens_dictionary[word]
        except:
            return []

        if (index > self.__distance_matrix.shape[0]):
            return []

        dist_order = np.argsort(self.__distance_matrix[index,:])[1:num_words+1]
        dist_list = self.__distance_matrix[index][dist_order]

        mask = np.ones_like(dist_list)
        mask = np.where(dist_list < delta)
        return [self.__inverse_tokens_dictionary[index] for index in dist_order[mask]]         
    
    ###### TO BE REDONE BETTER TO ACTUALLY REFORM A SENTENCE#####
    def __rejoin_review(self, sentences):
        new_sentences = sentences[:]
        for i, sent in enumerate (new_sentences):
            new_sentences[i] = ' '.join(sent)
        return ' '.join(new_sentences)
    
    def __perturb(self, sentences, word_index, neighbours, y_target):
        
        print('Starting perturbation:')
        
        prefix = ' '.join(sentences[word_index[0]][ : word_index[1]])
        suffix = ' '.join(sentences[word_index[0]][word_index[1]+1 : -1])
        
        if self.__top_neighbours > len(neighbours):
            top_neighbours = len(neighbours)
        else:
            top_neighbours = self.__top_neighbours
        
        lm_preds = self.__google_lm.get_words_probs(
            prefix, 
            neighbours[ : top_neighbours - 1], 
            suffix
        )
        
        #print(lm_preds)
        #print(np.argsort(lm_preds))
        
        score_list = []
        for adv_w in np.argsort(lm_preds)[-self.__top_lm : ]: 
            adv_sentences = sentences[:]
            adv_splitted_text = adv_sentences[word_index[0]][:]
            adv_splitted_text[word_index[1]] = neighbours[adv_w]
            adv_sentences[word_index[0]] = adv_splitted_text
            adv_review = self.__rejoin_review(adv_sentences)
            score = self.__black_box.predict_sentiment(adv_review)
            score_list += [(adv_review, score)]
            print(adv_review)
            print('Score:' + str(score) + '\n')
            
        adv_reviews_sorted =  sorted(score_list, key=lambda x: x[1])    
        
        print('Final review:')
        
        if y_target == 0:
            print(adv_reviews_sorted[0][0] + '\n')
            return adv_reviews_sorted[0]
        else:
            print(adv_reviews_sorted[-1][0] + '\n')
            return adv_reviews_sorted[-1]
        
        
    def __crossover(self, parent1, parent2):
        splitted_parent1 = parent1.split()
        splitted_parent2 = parent2.split()
        for i in range(len(splitted_parent1)):
            if np.random.uniform() < 0.5:
                splitted_parent1[i] = splitted_parent2[i]
        #print(splitted_parent1)
        return ' '.join(splitted_parent1)
        
        
    def attack (self, x_orig, y_orig):
        y_target = int(not y_orig)
        sentences = self.__split_review(x_orig)
        neighbours_dictionary = {}
        for sent_idx, sent in enumerate(sentences):
            for word_idx, word in enumerate(sent):
                neighbours_dictionary[(sent_idx, word_idx)] = self.__most_similar(
                    word = word.lower(), delta = 0.5, num_words = 50
                )
                
        #print(list(neighbours_dictionary.items())[:3])
        neighbours_length = {key: len(value) for key, value in neighbours_dictionary.items()}
        
        for key in neighbours_length.keys():
            if sentences[key[0]][key[1]].lower() in self.__stopwords:
                neighbours_length[key] = 0
                
        #print(list(neighbours_length.values())[:3])
        length_sum = sum(neighbours_length.values())
        
        #maybe optimize it
        neighbours_length = {key: value/length_sum for key, value in neighbours_length.items()}
        
        #for key, value in neighbours_length.items():
        #    print(sentences[key[0]][key[1]] + ' : ' + str(value))
        
        
        probabilities = list(neighbours_length.values())
        
        random_choice_size = len(np.nonzero(probabilities)[0]) if len(np.nonzero(probabilities)[0]) < self.__max_words else self.__max_words
        
        words_to_change = np.random.choice(
            len(neighbours_length.keys()), size = random_choice_size, replace = False, p = probabilities
        )
        
        print (words_to_change)
        
        print('Original sentence: \n' + x_orig + '\n')

        population = [self.__perturb(
                sentences, 
                list(neighbours_dictionary.keys())[index], 
                neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                y_target
            ) for index in words_to_change]
            
        for i in range(self.__max_gens):

            sorted_population =  sorted(population, key=lambda x: x[1])    

            #print('Final review:')

            if y_target == 0:
                best_attack = sorted_population[0]
            else:
                best_attack = sorted_population[-1]

            if round(best_attack[1]) == y_target:
                return best_attack[0]
        
            best_adversarials = [best_attack]
            
            pop_scores = np.array([score for review, score in sorted_population])
            
            print('Pop scores: ' + str(pop_scores))
            
            logits = np.exp(pop_scores - np.max(pop_scores))
            #logits = np.exp(pop_scores / 0.5)
            selection_probabilities = logits / np.sum(logits)
            
            print('Probab: ')
            print(selection_probabilities)
            
            if y_target == 0:
                selection_probabilities = selection_probabilities[::-1]
                print('New probab:')
                print(selection_probabilities)
            
            #if y_target == 0:
            #    selection_probabilities = [1 - score for review, score in sorted_population]
            #else:
            #    selection_probabilities = [score for review, score in sorted_population]
                
            parent_list_1 = np.random.choice(
                len(sorted_population), size=self.__max_children, p=selection_probabilities)
            parent_list_2 = np.random.choice(
                len(sorted_population), size=self.__max_children, p=selection_probabilities)
            
            children = [self.__crossover(
                sorted_population[parent_list_1[i]][0], 
                sorted_population[parent_list_2[i]][0]
            ) for i in range(self.__max_children)]
            
            perturbated_children = []
            
            for child in children:
                
                ######################################DUPLICATED CODE##############################################
                sentences = self.__split_review(child)
                neighbours_dictionary = {}
                for sent_idx, sent in enumerate(sentences):
                    for word_idx, word in enumerate(sent):
                        neighbours_dictionary[(sent_idx, word_idx)] = self.__most_similar(
                            word = word.lower(), delta = 0.5, num_words = 50
                        )

                #print(list(neighbours_dictionary.items())[:3])
                neighbours_length = {key: len(value) for key, value in neighbours_dictionary.items()}

                for key in neighbours_length.keys():
                    if sentences[key[0]][key[1]].lower() in self.__stopwords:
                        neighbours_length[key] = 0

                #print(list(neighbours_length.values())[:3])
                length_sum = sum(neighbours_length.values())

                #maybe optimize it
                neighbours_length = {key: value/length_sum for key, value in neighbours_length.items()}

                #for key, value in neighbours_length.items():
                #    print(sentences[key[0]][key[1]] + ' : ' + str(value))


                probabilities = list(neighbours_length.values())

                random_choice_size = len(np.nonzero(probabilities)[0]) if len(np.nonzero(probabilities)[0]) < self.__max_words else self.__max_words

                words_to_change = np.random.choice(
                    len(neighbours_length.keys()), size = random_choice_size, replace = False, p = probabilities
                )
                
                ######################################END DUPLICATED CODE##############################################
        
                perturbated_children += [self.__perturb(
                    sentences, 
                    list(neighbours_dictionary.keys())[index], 
                    neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                    y_target
                ) for index in words_to_change]
            
            population = [best_attack] + perturbated_children
            
        return None'''

'import string\nimport os\nimport numpy as np\nimport pickle\nimport licensed_scripts.lm_1b_eval as google_language_model_utils\nimport nltk\nfrom utils.black_box import BlackBox\n\nclass Attack(object):\n    \n    def __init__ (self):\n        with open(os.path.join(\'./pickle_data/attack_utils/tokens_dictionary.pickle\'), \'rb\') as f:\n            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)\n            self.__tokens_dictionary = tokens_dictionary\n            self.__inverse_tokens_dictionary = inverse_tokens_dictionary\n        f.close()\n        \n        self.__black_box = BlackBox()\n        self.__distance_matrix = np.load(os.path.join(\'./numpy_files/distance_matrix.npy\'))\n        self.__google_lm = google_language_model_utils.LM()\n        self.__stopwords = nltk.corpus.stopwords.words(\'english\')\n        self.__sentence_tokenizer = nltk.data.load(\'tokenizers/punkt/english.pickle\')\n        self.__tree_bank_word_tokenizer = nltk.tokenize.TreebankWordTo

In [96]:
'''import string
import os
import numpy as np
import pickle
import licensed_scripts.lm_1b_eval as google_language_model_utils
import nltk
from utils.black_box import BlackBox

class Attack(object):
    
    def __init__ (self):
        with open(os.path.join('./pickle_data/attack_utils/tokens_dictionary.pickle'), 'rb') as f:
            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)
            self.__tokens_dictionary = tokens_dictionary
            self.__inverse_tokens_dictionary = inverse_tokens_dictionary
        f.close()
        
        self.__black_box = BlackBox()
        self.__distance_matrix = np.load(os.path.join('./numpy_files/distance_matrix.npy'))
        self.__google_lm = google_language_model_utils.LM()
        self.__stopwords = nltk.corpus.stopwords.words('english')
        self.__sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        self.__tree_bank_word_tokenizer = nltk.tokenize.TreebankWordTokenizer()
        
        latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
        safe_characters = string.ascii_letters + string.digits + latin_similar + ' '
        safe_characters += "'"
        self.__safe_characters = safe_characters
        
        self.__max_iters = 10 # 20
        self.__top_neighbours = 20
        self.__top_lm = 5
        
        
    def __handle_contractions(self, sentence):
        sentence = self.__tree_bank_word_tokenizer.tokenize(sentence)
        return ' '.join(sentence)
    
    def __preprocess_sentence(self, sentence):
        chars = set(w for w in sentence)
        symbols = [c for c in chars if not c in self.__safe_characters]
        for symbol in symbols:
            sentence = sentence.replace(symbol, ' ' + symbol + ' ')
        sentence = self.__handle_contractions(sentence)
        return sentence.split()
    
    def __split_review(self, review):
        sentences = self.__sentence_tokenizer.tokenize(review)
        for index, sentence in enumerate(sentences):
            sentences[index] = self.__preprocess_sentence(sentence)
        return sentences
            
            
    def __most_similar(self, word, delta = 0.5, num_words = 20):
        try:
            index = self.__tokens_dictionary[word]
        except:
            return []

        if (index > self.__distance_matrix.shape[0]):
            return []

        dist_order = np.argsort(self.__distance_matrix[index,:])[1:num_words+1]
        dist_list = self.__distance_matrix[index][dist_order]

        mask = np.ones_like(dist_list)
        mask = np.where(dist_list < delta)
        return [self.__inverse_tokens_dictionary[index] for index in dist_order[mask]]         
    
    ###### TO BE REDONE BETTER TO ACTUALLY REFORM A SENTENCE#####
    def __rejoin_review(self, sentences):
        new_sentences = sentences[:]
        for i, sent in enumerate (new_sentences):
            new_sentences[i] = ' '.join(sent)
        return ' '.join(new_sentences)
    
    def __perturb(self, sentences, word_index, neighbours, y_target):        
        prefix = ' '.join(sentences[word_index[0]][ : word_index[1]])
        suffix = ' '.join(sentences[word_index[0]][word_index[1]+1 : -1])
        
        if self.__top_neighbours > len(neighbours):
            top_neighbours = len(neighbours)
        else:
            top_neighbours = self.__top_neighbours
        
        lm_preds = self.__google_lm.get_words_probs(
            prefix, 
            neighbours[ : top_neighbours - 1], 
            suffix
        )
        
        #print(lm_preds)
        #print(np.argsort(lm_preds))
        
        score_list = []
        for adv_w in np.argsort(lm_preds)[-self.__top_lm : ]: 
            adv_sentences = sentences[:]
            adv_splitted_text = adv_sentences[word_index[0]][:]
            adv_splitted_text[word_index[1]] = neighbours[adv_w]
            adv_sentences[word_index[0]] = adv_splitted_text
            adv_review = self.__rejoin_review(adv_sentences)
            score = self.__black_box.predict_sentiment(adv_review)
            score_list += [(adv_splitted_text, score)]
            
        adv_sentences_sorted =  sorted(score_list, key=lambda x: x[1])    
        
        if y_target == 0:
            final_adv_sentence = adv_sentences_sorted[0][0]
        else:
            final_adv_sentence = adv_sentences_sorted[-1][0]
            
        sentences[word_index[0]] = final_adv_sentence
        
        return sentences
        
        
    def attack (self, x_orig, y_orig):
        y_target = int(not y_orig)
        sentences = self.__split_review(x_orig)
        neighbours_dictionary = {}
        for sent_idx, sent in enumerate(sentences):
            for word_idx, word in enumerate(sent):
                neighbours_dictionary[(sent_idx, word_idx)] = self.__most_similar(
                    word = word.lower(), delta = 0.5, num_words = 50
                )
                
        #print(list(neighbours_dictionary.items())[:3])
        neighbours_length = {key: len(value) for key, value in neighbours_dictionary.items()}
        
        for key in neighbours_length.keys():
            if sentences[key[0]][key[1]] in self.__stopwords:
                neighbours_length[key] = 0
                
        #print(list(neighbours_length.values())[:3])
        length_sum = sum(neighbours_length.values())
        
        #maybe optimize it
        neighbours_length = {key: value/length_sum for key, value in neighbours_length.items()}
        
        words_to_change = np.random.choice(
            len(neighbours_length.keys()), size = self.__max_iters, p = list(neighbours_length.values())
        )
        
        print('Original sentence: \n' + x_orig + '\n')
        i=0
        
        for index in words_to_change:
            print('Iteration ' + str(i))
            i += 1
            sentences = self.__perturb(
                sentences, 
                list(neighbours_dictionary.keys())[index], 
                neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                y_target
            )
            
            print(self.__rejoin_review(sentences) + '\n')
            
        return self.__rejoin_review(sentences)
        
        
        
        
        '''

'import string\nimport os\nimport numpy as np\nimport pickle\nimport licensed_scripts.lm_1b_eval as google_language_model_utils\nimport nltk\nfrom utils.black_box import BlackBox\n\nclass Attack(object):\n    \n    def __init__ (self):\n        with open(os.path.join(\'./pickle_data/attack_utils/tokens_dictionary.pickle\'), \'rb\') as f:\n            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)\n            self.__tokens_dictionary = tokens_dictionary\n            self.__inverse_tokens_dictionary = inverse_tokens_dictionary\n        f.close()\n        \n        self.__black_box = BlackBox()\n        self.__distance_matrix = np.load(os.path.join(\'./numpy_files/distance_matrix.npy\'))\n        self.__google_lm = google_language_model_utils.LM()\n        self.__stopwords = nltk.corpus.stopwords.words(\'english\')\n        self.__sentence_tokenizer = nltk.data.load(\'tokenizers/punkt/english.pickle\')\n        self.__tree_bank_word_tokenizer = nltk.tokenize.TreebankWordTo

In [97]:
attack = Attack()

LM vocab loading done


Recovering graph.


INFO:tensorflow:Recovering Graph google_language_model\graph-2016-09-10.pbtxt


Recovering checkpoint google_language_model\ckpt-*


In [98]:
#del attack

In [99]:
adv_x0 = attack.attack(attack_list[4][0], attack_list[4][1])

Original sentence: 
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Generating population... 

Starting perturbation:


KeyboardInterrupt: 

In [54]:
attack_list[4][0]

"I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her."

In [55]:
adv_x0

"I think that the movie was frankly good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of females in farmers areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her."

In [56]:
black_box.predict_sentiment(adv_x0)

0.40761864

In [57]:
black_box.predict_sentiment(attack_list[4][0])

0.8942218

In [58]:
attack_list[4][1]

1

In [100]:
final_list = []
for i in range(5):
    print('########ATTACK {}/4##########:'.format(i))
    final_list += [(attack.attack(attack_list[i][0], attack_list[i][1]) , attack_list[i][1])]

########ATTACK 0/4##########:
Original sentence: 
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.

Generating population... 

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck cinematic choices. He work on a real dog with this one.

Score:0.056767795

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this individual.

Score:0.046122003

Starting perturbation:
Final review:
This film could cu

Final review:
This film could cure sleep disorders, thats how nasty it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film accomplished me wonder about Chuck cinematic choices. He work on a real dog with this one.

Score:0.2132879

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This cinematic made me wonder about Chuck cinematic choices. He work on a real dog with this one.

Score:0.09025106

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the naughty guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck cinematic choices. He work on a real dog with this one.

Score:0.12718247

Starting perturbation:
Final review:
This film could cure sleep disorders, thats 

Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film accomplished me wonder about Chuck cinematic choices. He job on a real dog with this one.

Score:0.18484266

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story pushed, and the bad guy is not that terrifying. You will not even see this one on TBS reruns. This film accomplished me wonder about Chuck cinematic choices. He work on a real dog with this one.

Score:0.32526726

Starting perturbation:
Final review:
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that terrifying. You will not even see this one on TBS reruns. This film accomplished me wonder about Chuck cinematic choice. He work on a real dog with this one.

Score:0.20358695

Starting perturbation:
Final review:
This film could treat sleep dis

Final review:
The trailers get you to the movie, but the movie just was n't worth my 8. 50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this one to their resume's. To be genuine, the book is better..

Score:0.109119184

Starting perturbation:
Final review:
The trailers get you to the movie, but the movie just was n't worth my 8. 50.. it has some best effects, but the narration.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this one to their resume's. To be honest, the book is better..

Score:0.16926977

Starting perturbation:
Final review:
The trailers get you to the movie, but the cinema just was n't worth my 8. 50.. it has some best effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this

Final review:
The trailers get you to the movie, but the movie just was n't worth my 8. 50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good cinema, but i wish they had n't added this one to their revival's. To be honest, the book is better..

Score:0.22389677

Starting perturbation:
Final review:
The trailers get you to the movie, but the movie just was n't worth my 8. 50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i desired they had n't added this one to their revival's. To be honest, the book is better..

Score:0.23349345

Starting perturbation:
Final review:
The trailers get you to the movie, but the movie just was n't worthy my 8. 50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added 

...

Final review:
greatest pile of fancy to date! Everyone involved with this production should be ashamed of themselves. Not one lonely element of the movie was anything somewhat loved an original idea. A premiere grader telling you a saga about nap time is more entertaining.

Score:0.5180556

Starting perturbation:
Final review:
greatest pile of fancy to date! Everyone involved with this production should be ashamed of themselves. Not one lonely aspect of the movie was anything somewhat loved an original idea. A premiere grader telling you a tale about nap time is more entertaining.

Score:0.7564107

Starting perturbation:
Final review:
greatest pile of fancy to date! Everyone involved with this production should be ashamed of themselves. Not one lonesome element of the movie was anything somewhat loved an original idea. A premiere grader telling you a tale about nap time is more entertaining.

Score:0.553772

Starting perturbation:
Final review:
greatest pile of fancy to date! all invol

Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. albeit the director has succeeded in showing the status of women in peasant region and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.8411289

Starting perturbation:
Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. albeit the director has succeeded in showing the status of women in rural region and how they suffering at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.88062495

Starting perturbation:
Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. albeit the director has avail in showing the status of women in rural region 

Final review:
I think that the movie was really good. matter, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's characters a bit and has focussed more on the violence faced by her.

Score:0.8563676

Starting perturbation:
Final review:
I thinks that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of women in rural areas and how they endure at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.8385451

Starting perturbation:
Final review:
I thinks that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the director has succeeded in showing the status of women in rural are

Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the chief has avail in showing the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.90023226

Starting perturbation:
Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's music were marvellous. Although the chief has succeeded in displaying the status of women in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.89869654

Starting perturbation:
Final review:
I think that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's musical were marvellous. Although the chief has succeeded in showing the status of women in rural areas and h

Final review:
I thinks that the movie was really good. topic, acting and Nusrat Fateh ALi Khan's musical were marvellous. albeit the director has succeeded in showing the status of female in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.8198603

Starting perturbation:
Final review:
I believes that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's musical were marvellous. albeit the director has succeeded in showing the status of female in rural areas and how they suffer at the hands of male-dominated culture, he has neglected Phoolan's character a bit and has focussed more on the violence faced by her.

Score:0.7958517

Starting perturbation:
Final review:
I thinks that the movie was really good. Subject, acting and Nusrat Fateh ALi Khan's song were marvellous. albeit the director has succeeded in showing the status of female in rural ar

In [101]:
print(final_list)

[('This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film accomplished me amazed about Chuck cinematic choices. He work on a real dog with this one.', 0), ("The trailers gets you to the movie, but the cinema just was n't worth my 8. 50.. it has some best effects, but the storyline.. yech, i like tommy lee jones and will smith as gamers, they have both finished some good films, but i wish they had n't added this individual to their revival's. To be honest, the book is better..", 0), ("The message of this movie is`` personality is more important than beauty''. Jeanine Garofalo is supposed to be the`` ugly duckling'', but the funny thing is that she's not at all ugly( actually she's a multiple more exciting than Uma Thurman, the boyfriend who looks love a model). Now, would this movie work if the`` ugly duckling'' was really repulsive? When will Hollywood stop with this hypocr

In [102]:
black_box.evaluate([sent for sent, lab in final_list], [lab for sent, lab in final_list])

5/5 [==============================] - 0s 31ms/sample - loss: 1.0768 - acc: 0.0000e+00


[1.0767607688903809, 0.0]

In [2]:
################################################################################################################################

In [247]:
import re
import nltk.data
import nltk


def attack(review, y_target):
    stopwords = nltk.corpus.stopwords.words('english')
    sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = sentence_tokenizer.tokenize(review)
    
    for sent_idx, sent in enumerate(sentences):
        if sent_idx < 3:
            splitted_text = sent.split()

            for w_idx, w in enumerate(splitted_text):
                if w_idx < 10:
                    w = w.lower()
                    pattern = r"[\w]+|[^\s\w]"
                    if len(re.findall(pattern, splitted_text[w_idx])) == 1 and w not in stopwords:
                        nearest_indexes = most_similar(w)
                        nearest_words = [inverse_tokens_dictionary[index] for index in nearest_indexes]
                        if len(nearest_words):
                            prefix = ' '.join(splitted_text[:w_idx])
                            print('Prefix:\n'+prefix)
                            suffix = ' '.join(splitted_text[w_idx + 1:])
                            print('Suffix:\n'+suffix)
                            lm_preds = google_language_model.get_words_probs(prefix, nearest_words, suffix)

                            score_list = []
                            for adv_w in np.argsort(lm_preds): 
                                adv_sentences = sentences[:]
                                adv_splitted_text = splitted_text[:]
                                adv_splitted_text[w_idx] = nearest_words[adv_w]
                                adv_sentences[sent_idx] = ' '.join(adv_splitted_text)
                                review_adv = ' '.join(adv_sentences)
                                score = black_box.predict_sentiment(review_adv)
                                score_list += [(adv_splitted_text, score)]
                                
                            adv_sentences_sorted =  sorted(score_list, key=lambda x: x[1])    
                            if y_target == 0:
                                splitted_text = adv_sentences_sorted[0][0]
                            else:
                                splitted_text = adv_sentences_sorted[-1][0]
                            sentences[sent_idx] = ' '.join(splitted_text)
    return ' '.join(sentences)

In [248]:
for i in range(5):
    x_orig = attack_list[i][0]
    y_orig = attack_list[i][1]
    
    y_target = int(not y_orig)
    
    adversarial_list.append((attack(x_orig, y_target), y_orig))
   
    
    
    print('Generated adversarial sentence')

Prefix:
This
Suffix:
could cure sleep disorders, thats how bad it is.
Prefix:
This movies
Suffix:
cure sleep disorders, thats how bad it is.
Prefix:
This movies did
Suffix:
sleep disorders, thats how bad it is.
Prefix:
This movies did remedy
Suffix:
disorders, thats how bad it is.
Prefix:
This movies did remedy hibernate disorders,
Suffix:
how bad it is.
Prefix:
This movies did remedy hibernate disorders, wouldnt how
Suffix:
it is.
Prefix:
The
Suffix:
dragged, and the bad guy is not that scary.
Prefix:
The tales dragged, and the
Suffix:
guy is not that scary.
Prefix:
The tales dragged, and the amiss
Suffix:
is not that scary.
Prefix:
You will not
Suffix:
see this one on TBS reruns.
Prefix:
You will not still
Suffix:
this one on TBS reruns.
Prefix:
You will not still admire this
Suffix:
on TBS reruns.
Generated adversarial sentence
Prefix:
The
Suffix:
get you to the movie, but the movie just wasn't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jone

In [214]:
#sorted(score_list, key=lambda x: x[1])[0][0]

'Wow. I observed this movie and "Up" on the same day within an hour of each other at different theaters. I saw "Mr Bug" first, and was then totally disappointed in "Up"\'s follow-up. What a beautiful and touching film! Movies of the 1930s and 40s to us nowadays can be irking with their melodramatic acting and dialog, but as animation the same melodrama and groaning humor can be wonderful. And the soft "organic" lines of 30s drawing AND the music just puts you in a nice comfortable mood and you can enjoy the show with all its little characters: ladybugs, grasshoppers, bees, snails, stinkbugs, flies, mosquitoes, beetles, crickets, and more each with all their own cute little (but not overbearing) idiosyncrasies. The interaction with the human world, from nemesis (cigar smokers, high-heel wearers, innocent kick-the-can playing kids) to the kind-hearted, and to the unknown destroyers, is realistic and fascinating. You care for the bugs, AND Dick and Mary. The protagonist Hoppity is not som

In [ ]:
#preds = google_language_model.get_words_probs('hey', ['how', 'kill', 'stupid'], 'are you')

In [206]:
#[i for i in np.argsort(preds)]

[1, 2, 0]

In [188]:
#preds

array([1.1748647e-04, 1.0273188e-05, 4.0947703e-05], dtype=float32)

In [249]:
adversarial_list[-5:]

[('This movies did remedy hibernate disorders, wouldnt how naughty it is. The tales dragged, and the amiss fella is not that scary. You will not still admire this uno on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.',
  0),
 ("The trailer obtains you to the movie, but the cinema just wasn't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they hadn't added this one to their resume's. To be honest, the ledger is better..",
  0),
 ('The voicemail of this cinema is "personality is more pivotal than beauty". janeane Garofalo is hypothetical to be the "ugly duckling", but the funny thing is that she\'s not at all ugly (actually she\'s a lot more attractive than Uma Thurman, the friend who looks like a model).Now, would this movie work if the "ugly duckling" was really unattractive? When will celebs halt with this hypocrisy?

In [254]:
attack_list[:5]

[('This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.',
  0),
 ("The trailers get you to the movie, but the movie just wasn't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they hadn't added this one to their resume's. To be honest, the book is better..",
  0),
 ('The message of this movie is "personality is more important than beauty". Jeanine Garofalo is supposed to be the "ugly duckling", but the funny thing is that she\'s not at all ugly (actually she\'s a lot more attractive than Uma Thurman, the friend who looks like a model).Now, would this movie work if the "ugly duckling" was really unattractive? When will Hollywood stop with this hypocrisy?In my opinion, despite the m

In [251]:
black_box.predict_all([sent for sent, lab in adversarial_list[-5:]])

[0.8820844292640686,
 0.22996748983860016,
 0.0038647179026156664,
 0.7400735020637512,
 0.39983099699020386]

In [252]:
black_box.evaluate([sent for sent, lab in adversarial_list[-5:]], [lab for sent, lab in adversarial_list[-5:]])

5/5 [==============================] - 0s 17ms/sample - loss: 0.9334 - acc: 0.4000


[0.9334100484848022, 0.4]

In [253]:
black_box.evaluate([sent for sent, lab in attack_list[:5]], [lab for sent, lab in attack_list[:5]])

5/5 [==============================] - 0s 30ms/sample - loss: 0.0490 - acc: 1.0000


[0.04902750998735428, 1.0]